In [1]:
import torch
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import music21 as m21
import pickle

from tqdm.auto import tqdm

from torch import nn,optim
from torch.utils.data import Dataset, DataLoader

print("Pytorch Version:",torch.__version__)
print("GPU Avaiable:",torch.cuda.is_available())

device = 'cuda' if torch.cuda.is_available() == True else 'cpu'
print("Setting device to:",device)


Pytorch Version: 1.9.0+cu102
GPU Avaiable: True
Setting device to: cuda


In [2]:
with open("/content/drive/MyDrive/Major Project/Final Dataset/inputs.pickle",'rb') as file:
  inputs = pickle.load(file)


In [3]:
with open("/content/drive/MyDrive/Major Project/Final Dataset/targets.pickle",'rb') as file:
  targets = pickle.load(file)

In [4]:
mapping = {
    "57": 0,
    "72": 1,
    "81": 2,
    "55": 3,
    "58": 4,
    "54": 5,
    "69": 6,
    "51": 7,
    "67": 8,
    "/": 9,
    "70": 10,
    "53": 11,
    "63": 12,
    "_": 13,
    "59": 14,
    "r": 15,
    "45": 16,
    "80": 17,
    "68": 18,
    "64": 19,
    "61": 20,
    "78": 21,
    "71": 22,
    "73": 23,
    "60": 24,
    "62": 25,
    "79": 26,
    "76": 27,
    "65": 28,
    "56": 29,
    "50": 30,
    "52": 31,
    "75": 32,
    "48": 33,
    "77": 34,
    "66": 35,
    "74": 36,
    "47": 37
}

In [5]:
input_size = len(mapping)
hidden_size = 256
num_layers = 2 
sequence_length = 64
learning_rate = 0.001
batch_size = 64

In [6]:
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, bidirectional=True)
        self.fc1 = nn.Linear(hidden_size * sequence_length*2, input_size)
        # self.fc2 = nn.Linear(hidden_size * 8, hidden_size * 2)
        # self.fc3 = nn.Linear(hidden_size *2 ,hidden_size)
        # self.fc4 = nn.Linear(hidden_size, input_size)
        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters(), learning_rate)
    
    def forward(self,x):
        
        lstm_output, _ = self.lstm(x )
        lstm_output = lstm_output.reshape(lstm_output.shape[0], -1)
        output1 = self.fc1(lstm_output)
        # output2 = self.fc2(output1)
        # output3 = self.fc3(output2)
        # output4 = self.fc4(output3)

        return output1

In [7]:
model = RNN().to(device)

In [8]:
class Music_Dataset(Dataset):
  def __init__(self, music, label):
    self.music = music
    self.label = label
    self.n_samples = music.shape[0]

  def __getitem__(self, index):
    return self.music[index], self.label[index]

  def __len__(self):
    return self.n_samples

Data = Music_Dataset(inputs, targets)
train_loader = DataLoader(Data, batch_size = batch_size, shuffle = False, drop_last = True)

In [ ]:
# start training
for i in tqdm(range(100)):
    for x,y in tqdm(train_loader):
        x = x.to(device)
        y = y.to(device)
        model.optimizer.zero_grad()
        outputs = model(x)
        #print(outputs)
        #print(outputs.shape)
        loss = model.criterion(outputs,y)
        #print(loss)

        loss.backward()
        model.optimizer.step()


In [9]:
PATH = "/content/drive/MyDrive/Major Project/Model/melody_generator.pth"


In [ ]:
torch.save(model.state_dict(), PATH)

In [10]:
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [11]:
model.eval()

RNN(
  (lstm): LSTM(38, 256, num_layers=2, batch_first=True, bidirectional=True)
  (fc1): Linear(in_features=32768, out_features=38, bias=True)
  (criterion): CrossEntropyLoss()
)

In [90]:
class MelodyGenerator:
    """A class that wraps the LSTM model and offers utilities to generate melodies."""

    def __init__(self):
        """Constructor that initialises TensorFlow model"""

        #self.model_path = model_path
        self.model = model

        # with open(MAPPING_PATH, "r") as fp:
        #     self._mappings = json.load(fp)

        self._mappings = mapping
        self._start_symbols = ["/"] * sequence_length



    def generate_melody(self, seed, num_steps, max_sequence_length):
        """Generates a melody using the DL model and returns a midi file.
        :param seed (str): Melody seed with the notation used to encode the dataset
        :param num_steps (int): Number of steps to be generated
        :param max_sequence_len (int): Max number of steps in seed to be considered for generation
        :return melody (list of str): List with symbols representing a melody
        """

        # create seed with start symbols
        seed = seed.split()
        melody = seed
        seed = self._start_symbols + seed

        # map seed to int
        seed = [self._mappings[symbol] for symbol in seed]

        for _ in range(num_steps):

            # limit the seed to max_sequence_length
            seed = seed[-max_sequence_length:]

            # one-hot encode the seed
            
            onehot_seed = F.one_hot(torch.tensor(seed), num_classes=len(self._mappings)).numpy()
            # (1, max_sequence_length, num of symbols in the vocabulary)
            
            onehot_seed = onehot_seed[np.newaxis, ...]

            onehot_seed_tensor = torch.Tensor(onehot_seed).to(device)
            # make a prediction
            

            predictions = self.model(onehot_seed_tensor)[0]

            output_int = int(predictions.topk(1)[1])
            # update seed
            seed.append(output_int)

            # map int to our encoding
            output_symbol = [k for k, v in self._mappings.items() if v == output_int][0]

            # check whether we're at the end of a melody
            if output_symbol == "/":
                break
                #continue
            # update melody
            melody.append(output_symbol)

        return melody


    def save_melody(self, melody, step_duration=0.25, format="midi", file_name="mel.mid"):
        """Converts a melody into a MIDI file
        :param melody (list of str):
        :param min_duration (float): Duration of each time step in quarter length
        :param file_name (str): Name of midi file
        :return:
        """

        # create a music21 stream
        stream = m21.stream.Stream()

        start_symbol = None
        step_counter = 1

        # parse all the symbols in the melody and create note/rest objects
        for i, symbol in enumerate(melody):

            # handle case in which we have a note/rest
            if symbol != "_" or i + 1 == len(melody):

                # ensure we're dealing with note/rest beyond the first one
                if start_symbol is not None:

                    quarter_length_duration = step_duration * step_counter # 0.25 * 4 = 1

                    # handle rest
                    if start_symbol == "r":
                        m21_event = m21.note.Rest(quarterLength=quarter_length_duration)

                    # handle note
                    else:
                        m21_event = m21.note.Note(int(start_symbol), quarterLength=quarter_length_duration)

                    stream.append(m21_event)

                    # reset the step counter
                    step_counter = 1

                start_symbol = symbol

            # handle case in which we have a prolongation sign "_"
            else:
                step_counter += 1

        # write the m21 stream to a midi file
        stream.write(format, file_name)






In [91]:
mg = MelodyGenerator()
seed = "71 _ 72 _ 71 _ 69 _ 69 _ 71 _ 72 _ 74 _ 72 _ 71 _ _ _ 72 _ 74 _ 76 _ _ _ 76 _ 76 _ 76 78 76 74 76 _ _ _ r _ 76 _ 76 _ 74 _ 72 _ _ _ 76 _ _ _ 74 _ 72 _ 71 _ _ _ 69 _ 71 _ 71 _ 72 _ 71 _"
seed2 = "65 _ 64 _ 62 _ 60 _ _ _"
seed3 = '71 _ '
seed4 = "55 _ 55 _ 59 _ 60 _ _ _ 64 _ _ _ 64 _ 62 _ 62 _ 60 _ r _ 60 _ 60 _ 64 _ 67 _ _ _ 67 _ _ _ 67 _ 65 _ 65 _ 64 _ r _ _ _ _ _ 60 _ 60 _ 69 _ 69 _ 69 _ 69 _ 67 _ 67 _ _ _ 67 _ 65 _ 69 _ 65 _ 65 _ 64 _ 67 _ 60 _ 60 _ 69 _ 69 _ 69 _ 69 _ 67 _ 67 _ _ _ 67 _ 65 _ 69 _ 65 _ 65 _ 64 _ 67 _ _ _ 62 62 62 62 65 _ _ _ 64 64 64 64 67 _ _ _ 64 _ 65 _ 67 _ _ 69 67 _ 65 _ 64 _ 62 _ 60 _ _ _ r _ _ _ 62 62 62 62 65 _ _ _ 64 64 64 64 67 _ _ _ "
seed5 = "71 _ 72 _ 71 _ 69 _ 69 _ 71 _ 72 _ 74 _ 72 _ 71"

seed6 = "72 _ 74 _ 72 _ 71"
seed7 = "71 _ 72 _ 71 _ 69 _ 69 _ 71 _ 72 _ 74 _ 72 _ 71 _ _ _ 72 _ 74 _ 76 _ _ _ 76 _ 76 _ 76 78 76 "
seed8 = "69"

In [92]:
melody = mg.generate_melody(seed8, 1000, sequence_length)
print(melody)
mg.save_melody(melody)

['69', '_', '71', '_', '72', '_', '74', '_', '72', '_', '71', '_', '69', '_', '_', '_', '71', '_', '72', '_', '74', '_', '76', '_', '74', '_', '72', '_', '71', '_', '_', '_', '74', '_', '_', '_', '76', '_', '_', '_', '76', '_', '_', '_', '69', '_', '_', '_', '74', '_', '_', '_', '72', '_', '_', '_', '71', '_', '_', '_', '69', '_', '_', '_', '69', '_', '_', '_', '76', '_', '_', '_', '_', '_', '74', '_', '76', '_', '_', '_', '77', '_', '_', '_', '74', '_', '_', '_', '_', '_', '72', '_', '74', '_', '_', '_', '76', '_', '_', '_', '72', '_', '_', '_', '_', '_', '71', '_', '69', '_', '_', '_', '74', '_', '_', '_', '72', '_', '_', '_', '71', '_', '_', '_', '69', '_', '_', '_']
